In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_med_df,
    get_visualization_subtitle,
    apply_theme,
    get_anonymousid_color_maps
)
from web import for_website

alt.data_transformers.disable_max_rows() # Allow using rows more than 5000

In [ ]:
df = read_full_med_df()
df.head()

In [ ]:
def preprocess_df(df):
    
    # Negative values to zeros
    df.loc[df['num_patients_all_before_admission'] < 0, 'num_patients_all_before_admission'] = 0
    df.loc[df['num_patients_all_since_admission'] < 0, 'num_patients_all_since_admission'] = 0
    df.loc[df['num_patients_ever_severe_before_admission'] < 0, 'num_patients_ever_severe_before_admission'] = 0
    df.loc[df['num_patients_ever_severe_since_admission'] < 0, 'num_patients_ever_severe_since_admission'] = 0
    
    # Wide to long
    df = pd.melt(df, id_vars=[
        'siteid', 
        'med_class'
    ])
    df = df.rename(columns={"variable": 'category', "value": 'num_patients'})
    
    df['siteid'] = df['siteid'].apply(lambda x: get_siteid_anonymous_map()[x])
    
    return df

In [ ]:
pdf = preprocess_df(df)
pdf.head()

In [ ]:
unique_med_class = pdf['med_class'].unique().tolist()
input_dropdown = alt.binding_select(options=unique_med_class)
selection = alt.selection_single(fields=['med_class'], bind=input_dropdown, name='Med Class ', init={'med_class': unique_med_class[0]})
# legend_selection = alt.selection_multi(fields=["Country"], bind="legend")

# COUNTRIES = ["All countries", "France", "Germany", "Italy", "USA", "Within site variation"]
# COUNTRY_COLOR = ["#444444", "#0072B2", "#E69F00", "#009E73", "#D55E00", "#444444"]
# color_scale = alt.Scale(domain=COUNTRIES, range=COUNTRY_COLOR)
tick_size = 40

base = alt.Chart(pdf).mark_bar(size=tick_size).encode(
#     x=alt.Y('Country:N'),
#     color=alt.Color('Country:N', scale=color_scale, title=None),
#     stroke=alt.Color('Country:N', scale=color_scale),
    strokeWidth=alt.value(1),
    y=alt.Y('min(num_patients_ever_severe_before_admission):Q', title='Mean (SD)'),
    y2=alt.Y2('max(num_patients_ever_severe_before_admission):Q'),
#     tooltip=[
#         alt.Tooltip("Country", title="Category"),
#         alt.Tooltip("mean_val", title="Mean", format=".2f"),
#         alt.Tooltip("stdev_val", title="Standard deviation", format=".2f"),
#         alt.Tooltip("days_since_positive", title="Days since positive")
#     ]
)

mean = alt.Chart(pdf).mark_tick(size=tick_size, thickness=2).encode(
#     x=alt.X('Country:N', title=None),
    opacity=alt.value(1),
    color=alt.value('white'),
    y=alt.Y('mean(num_patients_ever_severe_before_admission):Q')
)

plot = (
    alt.layer(base, mean)
    .add_selection(selection)
#     .add_selection(legend_selection)
    .transform_filter(selection)
#     .transform_filter(legend_selection)
#     .transform_filter(alt.FieldOneOfPredicate(field='Country', oneOf=COUNTRIES))
#     .transform_calculate(y='datum.mean_val + datum.stdev_val',y2='datum.mean_val - datum.stdev_val')
    .properties(
        title={
            "text": ["Medications"], 
            "dx": 50,
            "subtitle": get_visualization_subtitle(),
            "subtitleColor": "gray",
            "anchor": "middle",
        },
        width=350, height=400
    )
)



plot = apply_theme(plot)

# for_website(plot, "Labs", "Lab variation across sites on Day 0")

plot